## `fastplotlib` demo

In [1]:
import zmq
import numpy as np
import fastplotlib as fpl

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01,\x00\x00\x007\x08\x06\x00\x00\x00\xb6\x1bw\x99\x…

No windowing system present. Using surfaceless platform
No config found!
No config found!


Available devices:
✅ (default) | NVIDIA GeForce GTX 1080 Ti | DiscreteGPU | Vulkan | 555.42.06
❗ | llvmpipe (LLVM 15.0.7, 256 bits) | CPU | Vulkan | Mesa 23.2.1-1ubuntu3.1~22.04.2 (LLVM 15.0.7)
❗ | NVIDIA GeForce GTX 1080 Ti/PCIe/SSE2 | Unknown | OpenGL | 


In [2]:
# ZMQ context and socket setup
context = zmq.Context()
socket = context.socket(zmq.SUB)
socket.connect("tcp://127.0.0.1:5555")
socket.setsockopt_string(zmq.SUBSCRIBE, "")

In [3]:
print("Listening...")
# Create the figure
figure = fpl.Figure()

# Add a line plot placeholder
xs = np.linspace(-10, 10, 100)
cos_ys = np.cos(xs)
figure[0, 0].add_line(data=np.column_stack((xs, cos_ys)), name="wave")

Listening...


RFBOutputContext()

/home/edwin/edwin_improv/fpl/lib/python3.10/site-packages/fastplotlib/graphics/_features/_base.py:21: UserWarning: casting float64 array to float32
  warn(f"casting {array.dtype} array to float32")


<weakproxy at 0x70af7bf18e50 to LineGraphic at 0x70af80864fa0>

In [4]:
def get_buffer():
    """
    Retrieve the buffer from the socket.
    """
    try:
        b = socket.recv(zmq.NOBLOCK)  # Non-blocking receive
        return b
    except zmq.Again:
        return None

In [5]:
def update_frame(p):
    """
    Update the frame using 2D data received from the socket and add z-coordinates.
    """
    buff = get_buffer()
    if buff is not None:
        # Deserialize the buffer into a NumPy array
        data = np.frombuffer(buff, dtype=np.float64)

        # Extract the frame number and 2D values
        frame_num = int(data[0])  # First element is the frame number
        values = data[1:].reshape(-1, 2)  # Reshape the remaining data into 2D array [x, y]

        # Update the line plot
        p["wave"].data[:, :2] = values
        p.name = f"frame: {frame_num}"

In [6]:
# Add the animation update function
figure[0, 0].add_animations(update_frame)

figure.show()

JupyterOutputContext(children=(JupyterWgpuCanvas(), IpywidgetToolBar(children=(Button(icon='expand-arrows-alt'…